In [141]:
# IMPORTS

import pandas as pd
import numpy as np

L’identification des acteurs les plus présents et les périodes associées 
Dans la table title.principals.tsv.gz on récupère toutes les lignes category == 'actor' 
On récupère de l'autre côté la table title.basics.tsv.gz avec que les films

BASES A CHARGER

In [109]:
# On charge la bd title.principals et on remplace les \N par NaN
df_acteur = pd.read_csv('BD/title.principals.tsv.gz', compression = 'gzip', sep = '\t')
df_acteur = df_acteur[(df_acteur['category'] == 'actor') | (df_acteur['category'] == 'actress') | (df_acteur['category'] == 'self')]
# df_acteur.replace('\\N', np.nan , inplace = True)

In [150]:
len(df_acteur)

50137648

In [110]:
# On charge la bd title.basics et on remplace les \N par NaN

df_film = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')
df_film = df_film[df_film['titleType'] == 'movie']
# df_film['startYear'].replace('\\N', np.nan, inplace = True)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_7985/2584169011.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_film = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')


In [111]:
# On charge la bd name.basics et on remplace les \N par NaN
 
df_nom = pd.read_csv('BD/name.basics.tsv.gz', compression = 'gzip', sep = '\t')
# df_nom.replace('\\N', np.nan, inplace = True)

In [112]:
'''
df_intermediaire -> table des acteurs présents dans un film
df_film -> table des films
df_nom -> table des noms des acteurs
'''

'\ndf_intermediaire -> table des acteurs présents dans un film\ndf_film -> table des films\ndf_nom -> table des noms des acteurs\n'

In [113]:
print(len(df_acteur))
print(len(df_film))
print(len(df_nom))

50137648
697513
13956187


MERGES

In [154]:
# MERGE Acteurs et Films INNER

df_acteur_film_inner = pd.merge(df_acteur,
                                df_film,
                                how = 'inner',
                                on = 'tconst') 

In [155]:
# MERGE Acteurs et Films OUTER

df_acteur_film_outer = pd.merge(df_acteur,
                                df_film,
                                how = 'outer',
                                on = 'tconst') 

In [156]:
# MERGE Acteurs et Films RIGHT

df_acteur_film_right = pd.merge(df_acteur,
                                df_film,
                                how = 'right',
                                on = 'tconst') 

In [158]:
# MERGE Acteurs et Films LEFT

df_acteur_film_left = pd.merge(df_acteur,
                                df_film,
                                how = 'left',
                                on = 'tconst') 

In [159]:
print(len(df_acteur_film_inner))
print(len(df_acteur_film_outer))
print(len(df_acteur_film_left))
print(len(df_acteur_film_right))

4162172
50284421
50137648
4308945


Nombre d'apparitions

In [116]:
# On compte le nombre d'occurence de chaque acteur dans ce nouveau df

df_vc = df_acteur_film_inner['nconst'].value_counts()

In [148]:
len(df_vc)

1445430

In [117]:
# On charge le df dans lequel le nom des acteurs est contenu et on le le merge avec notre df du nombre d'occurrence pour obtenir un acteur et son nb d'occurence

df_apparition = pd.merge(df_vc,
                          df_nom,
                          how = 'left',
                          on = 'nconst') 

In [147]:
len(df_apparition)

1445430

MIN et MAX de STARTYEAR

In [128]:
# On supprime les lignes où StartYear est vide ou est égal à \N

df_acteur_film_outer = df_acteur_film_outer[~(df_acteur_film_outer['startYear'] == '\\N')]
df_acteur_film_outer = df_acteur_film_outer[~(df_acteur_film_outer['startYear'].isna())]

In [129]:
# On supprime les lignes où NCONST est vide

df_acteur_film_outer = df_acteur_film_outer[~(df_acteur_film_outer['nconst'].isna())]

In [130]:
# On crée deux colonne une dans laquel on stock l'année minimum et l'autre max

df_min = df_acteur_film_outer.groupby('nconst')['startYear'].min()
df_max = df_acteur_film_outer.groupby('nconst')['startYear'].max()

In [131]:
# On merge les dates MIN et MAX de StartYear

df_dates = pd.merge(df_max,
                    df_min,
                    how = 'outer',
                    on = 'nconst') 

In [132]:
# On merge df_acteur_film avec le Min de StartYear et le Max de StartYear

df_kpi1 = pd.merge(df_apparition,
                   df_dates,
                   how = 'left',
                   on = 'nconst') 

In [133]:
# Rename des colonnes pour y voir clair

df_kpi1 = df_kpi1.rename(columns = {'startYear_y' : 'minStartYear','startYear_x' : 'maxStartYear'})

In [134]:
df_kpi1 = df_kpi1[['primaryName', 'count', 'minStartYear', 'maxStartYear']]
df_kpi1 = df_kpi1.rename(columns = {'primaryName' : 'Acteur/Actrice','count' : 'Nombre de films',
                                          'minStartYear' : 'Premier film', 'maxStartYear' : 'Dernier film'})

In [135]:
df_kpi1

,Acteur/Actrice,Nombre de films,Premier film,Dernier film
0,Brahmanandam,1131,1987,2025
1,Jagathy Sreekumar,661,1957,2023
2,Adoor Bhasi,626,1953,1991
3,Paquito Diaz,622,1958,2005
4,Eddie Garcia,616,1949,2019
...,...,...,...,...
1445425,Kayleigh Lawrence,1,2009,2009
1445426,Chelsea Dixon,1,2009,2009
1445427,Luke Emsley,1,2009,2009
1445428,Teijun Ogawa,1,2008,2008
